In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
'''
FEATURES:
Overall action count/ratio          DONE
Overall day count                   DONE
Monthly action count/ratio          DONE
Product Diversity                   DONE
Penetration                     TODO
Monthly Aggregation             TODO
User Aggregation                TODO
Double 11 Features                  DONE
Latest One-Week                     DONE
LDA Features                        DONE
PCA Features                        DONE
Age Related                         DONE
Gender Related                      DONE

'''

'\nFEATURES:\nOverall action count/ratio          DONE\nOverall day count                   DONE\nMonthly action count/ratio          DONE\nProduct Diversity                   DONE\nPenetration                     TODO\nMonthly Aggregation             TODO\nUser Aggregation                TODO\nDouble 11 Features                  DONE\nLatest One-Week                     DONE\nLDA Features                        DONE\nPCA Features                        DONE\nAge Related                         DONE\nGender Related                      DONE\n\n'

In [2]:
df = pd.read_csv("./use_data/expanded_testing.csv")
df

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,age_range,gender,prob
0,149002,282,885,3791,1500.0,1111,0,5.0,2.0,NaN
1,149002,282,885,3791,1500.0,1107,0,5.0,2.0,NaN
2,109881,230,629,1,1662.0,820,0,4.0,0.0,NaN
3,109881,230,629,1,1662.0,820,0,4.0,0.0,NaN
4,109881,230,629,1,1662.0,820,0,4.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...
6544,73105,259,614,637,2603.0,1111,0,2.0,0.0,NaN
6545,403421,175,1181,4760,247.0,1111,0,3.0,1.0,NaN
6546,116738,186,267,1200,2276.0,1110,0,2.0,1.0,NaN
6547,116738,186,267,1200,2276.0,1101,0,2.0,1.0,NaN


In [3]:
df['gender'] = df['gender'].fillna(2.0)
df['age_range'] = df['age_range'].fillna(0)
df = df[df['brand_id'].notna()]

In [4]:
# seller == merchant
df_merchant = df[['seller_id']]
df_merchant

,seller_id
0,3791
1,3791
2,1
3,1
4,1
...,...
6544,637
6545,4760
6546,1200
6547,1200


In [5]:
# Overall action count/ratio
actions = {}

# go through user_seller sub-dataframe and split entries by user_id:seller_id
for index, row in df.iterrows():
    if row['seller_id'] not in actions:
        # there may be multiple entries, so default with an empty list
        actions[row["seller_id"]] = [] 
    actions[row["seller_id"]].append(row["action_type"])

In [6]:
actions

{3791.0: [0.0,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 1.0: [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 2781.0: [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  2.0,
  3.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.

In [7]:
#for k,v in actions.items():
#    actions[k] = list(set(v)) # make each value list unique
#actions

# same amount?

In [8]:
action_count = {}
for key in actions.keys():
    action_count[key] = len(actions[key])
action_count

{3791.0: 14,
 1.0: 45,
 2781.0: 96,
 3989.0: 1523,
 3096.0: 66,
 158.0: 23,
 1765.0: 26,
 4257.0: 307,
 4955.0: 58,
 3828.0: 124,
 3101.0: 20,
 3323.0: 413,
 3083.0: 30,
 1913.0: 29,
 145.0: 111,
 4798.0: 237,
 834.0: 31,
 1641.0: 268,
 4282.0: 60,
 831.0: 14,
 637.0: 885,
 2623.0: 12,
 4760.0: 398,
 2642.0: 2,
 4321.0: 134,
 3766.0: 75,
 2823.0: 5,
 4205.0: 23,
 2871.0: 3,
 3196.0: 40,
 139.0: 30,
 2387.0: 61,
 801.0: 72,
 615.0: 12,
 1362.0: 2,
 3673.0: 12,
 2925.0: 84,
 3491.0: 20,
 4882.0: 23,
 246.0: 20,
 4129.0: 24,
 1131.0: 42,
 419.0: 73,
 2085.0: 54,
 3003.0: 10,
 4138.0: 26,
 3939.0: 4,
 1598.0: 20,
 4224.0: 2,
 623.0: 18,
 2091.0: 15,
 4440.0: 8,
 780.0: 13,
 1784.0: 30,
 524.0: 7,
 3119.0: 3,
 2292.0: 3,
 2159.0: 21,
 4744.0: 3,
 582.0: 4,
 4964.0: 1,
 1087.0: 36,
 1985.0: 5,
 4218.0: 28,
 2560.0: 12,
 4549.0: 10,
 3751.0: 9,
 3769.0: 9,
 4597.0: 8,
 3246.0: 4,
 2960.0: 13,
 4538.0: 60,
 644.0: 40,
 5.0: 2,
 461.0: 7,
 1943.0: 31,
 314.0: 27,
 630.0: 23,
 2277.0: 72,
 1210.

In [9]:
# total action count
total_action_count = sum(action_count.values())
total_action_count

6546

In [10]:
action_count_ratio = {}
for key in action_count:
    action_count_ratio[key] = action_count[key]/total_action_count
action_count_ratio

{3791.0: 0.0021387106630003055,
 1.0: 0.0068744271310724105,
 2781.0: 0.01466544454628781,
 3989.0: 0.23266116712496182,
 3096.0: 0.01008249312557287,
 158.0: 0.003513596089214788,
 1765.0: 0.003971891231286282,
 4257.0: 0.04689886953864956,
 4955.0: 0.008860372746715552,
 3828.0: 0.018942865872288422,
 3101.0: 0.0030553009471432934,
 3323.0: 0.06309196455850902,
 3083.0: 0.00458295142071494,
 1913.0: 0.004430186373357776,
 145.0: 0.01695692025664528,
 4798.0: 0.03620531622364803,
 834.0: 0.0047357164680721054,
 1641.0: 0.040941032691720135,
 4282.0: 0.00916590284142988,
 831.0: 0.0021387106630003055,
 637.0: 0.13519706691109074,
 2623.0: 0.0018331805682859762,
 4760.0: 0.06080048884815154,
 2642.0: 0.00030553009471432935,
 4321.0: 0.020470516345860067,
 3766.0: 0.011457378551787351,
 2823.0: 0.0007638252367858234,
 4205.0: 0.003513596089214788,
 2871.0: 0.00045829514207149406,
 3196.0: 0.006110601894286587,
 139.0: 0.00458295142071494,
 2387.0: 0.009318667888787045,
 801.0: 0.01099908

In [11]:
df_merchant['merchant action count'] = -1
df_merchant['merchant action count ratio'] = -1
for index,row in df_merchant.iterrows():
        df_merchant.at[index, 'merchant action count'] = action_count[row['seller_id']]
        df_merchant.at[index, 'merchant action count ratio'] = action_count_ratio[row['seller_id']]

df_merchant

/tmp/ipykernel_17092/345860918.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merchant['merchant action count'] = -1
/tmp/ipykernel_17092/345860918.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merchant['merchant action count ratio'] = -1


,seller_id,merchant action count,merchant action count ratio
0,3791,14,0.002139
1,3791,14,0.002139
2,1,45,0.006874
3,1,45,0.006874
4,1,45,0.006874
...,...,...,...
6544,637,885,0.135197
6545,4760,398,0.060800
6546,1200,13,0.001986
6547,1200,13,0.001986


In [12]:
#FEATURE 2: OVERALL DAY COUNT
days = {}

# go through user_seller sub-dataframe and split entries by user_id:seller_id
for index, row in df.iterrows():
    if row['seller_id'] not in days:
        # there may be multiple entries, so default with an empty list
        days[row["seller_id"]] = [] 
    days[row["seller_id"]].append(row["time_stamp"])

In [13]:
for k,v in days.items():
    days[k] = list(set(v)) # make each value list unique
days

{3791.0: [708.0,
  906.0,
  1008.0,
  531.0,
  1107.0,
  1011.0,
  1108.0,
  1111.0,
  603.0,
  604.0,
  605.0],
 1.0: [516.0,
  903.0,
  904.0,
  905.0,
  906.0,
  523.0,
  909.0,
  528.0,
  912.0,
  914.0,
  915.0,
  531.0,
  917.0,
  920.0,
  921.0,
  923.0,
  803.0,
  819.0,
  820.0,
  828.0,
  830.0,
  702.0,
  704.0,
  705.0,
  706.0,
  707.0,
  709.0,
  710.0,
  711.0,
  715.0,
  718.0,
  728.0,
  730.0,
  614.0,
  617.0,
  1017.0],
 2781.0: [1028.0,
  1101.0,
  1102.0,
  1103.0,
  1104.0,
  1105.0,
  1107.0,
  1109.0,
  1110.0,
  1111.0,
  927.0],
 3989.0: [512.0,
  1025.0,
  1024.0,
  1027.0,
  516.0,
  517.0,
  1030.0,
  1031.0,
  1028.0,
  521.0,
  519.0,
  523.0,
  522.0,
  525.0,
  526.0,
  518.0,
  528.0,
  530.0,
  531.0,
  1026.0,
  1102.0,
  1103.0,
  1104.0,
  1105.0,
  1106.0,
  1107.0,
  1108.0,
  1109.0,
  1110.0,
  1111.0,
  601.0,
  602.0,
  603.0,
  604.0,
  605.0,
  607.0,
  609.0,
  610.0,
  611.0,
  612.0,
  613.0,
  614.0,
  615.0,
  616.0,
  617.0,
  618.0,

In [14]:
day_count = {}
for key in days.keys():
    day_count[key] = len(days[key])
day_count

{3791.0: 11,
 1.0: 36,
 2781.0: 11,
 3989.0: 163,
 3096.0: 22,
 158.0: 6,
 1765.0: 14,
 4257.0: 117,
 4955.0: 9,
 3828.0: 45,
 3101.0: 13,
 3323.0: 57,
 3083.0: 16,
 1913.0: 13,
 145.0: 11,
 4798.0: 30,
 834.0: 6,
 1641.0: 67,
 4282.0: 31,
 831.0: 8,
 637.0: 149,
 2623.0: 6,
 4760.0: 36,
 2642.0: 2,
 4321.0: 71,
 3766.0: 28,
 2823.0: 5,
 4205.0: 15,
 2871.0: 3,
 3196.0: 13,
 139.0: 19,
 2387.0: 7,
 801.0: 33,
 615.0: 8,
 1362.0: 1,
 3673.0: 9,
 2925.0: 45,
 3491.0: 12,
 4882.0: 11,
 246.0: 8,
 4129.0: 18,
 1131.0: 8,
 419.0: 42,
 2085.0: 21,
 3003.0: 7,
 4138.0: 11,
 3939.0: 2,
 1598.0: 10,
 4224.0: 2,
 623.0: 8,
 2091.0: 10,
 4440.0: 6,
 780.0: 10,
 1784.0: 7,
 524.0: 6,
 3119.0: 2,
 2292.0: 3,
 2159.0: 17,
 4744.0: 3,
 582.0: 4,
 4964.0: 1,
 1087.0: 6,
 1985.0: 4,
 4218.0: 6,
 2560.0: 2,
 4549.0: 7,
 3751.0: 6,
 3769.0: 4,
 4597.0: 6,
 3246.0: 4,
 2960.0: 10,
 4538.0: 4,
 644.0: 28,
 5.0: 2,
 461.0: 5,
 1943.0: 6,
 314.0: 5,
 630.0: 16,
 2277.0: 8,
 1210.0: 7,
 3923.0: 1,
 4185.0: 10

In [15]:
df_merchant['merchant day count'] = -1
for index,row in df_merchant.iterrows():
        df_merchant.at[index, 'merchant day count'] = day_count[row['seller_id']]

df_merchant

/tmp/ipykernel_17092/3594710395.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merchant['merchant day count'] = -1


,seller_id,merchant action count,merchant action count ratio,merchant day count
0,3791,14,0.002139,11
1,3791,14,0.002139,11
2,1,45,0.006874,36
3,1,45,0.006874,36
4,1,45,0.006874,36
...,...,...,...,...
6544,637,885,0.135197,149
6545,4760,398,0.060800,36
6546,1200,13,0.001986,4
6547,1200,13,0.001986,4


In [16]:
#FEATURE 3: Monthly ACTION/COUNT ratio


In [17]:
#FEATURE 4: Product Diversity
items = {}

# go through user_seller sub-dataframe and split entries by user_id:seller_id
for index, row in df.iterrows():
    if row['seller_id'] not in items:
        # there may be multiple entries, so default with an empty list
        items[row["seller_id"]] = [] 
    items[row["seller_id"]].append(row["item_id"])

In [18]:
for k,v in items.items():
    items[k] = list(set(v)) # make each value list unique
items

{3791.0: [282.0],
 1.0: [230.0],
 2781.0: [193.0, 182.0],
 3989.0: [304.0],
 3096.0: [217.0],
 158.0: [257.0],
 1765.0: [212.0],
 4257.0: [174.0],
 4955.0: [224.0],
 3828.0: [170.0, 290.0],
 3101.0: [265.0],
 3323.0: [279.0],
 3083.0: [309.0],
 1913.0: [210.0],
 145.0: [184.0, 198.0],
 4798.0: [294.0],
 834.0: [239.0],
 1641.0: [178.0],
 4282.0: [277.0],
 831.0: [302.0],
 637.0: [259.0],
 2623.0: [256.0],
 4760.0: [175.0],
 2642.0: [315.0],
 4321.0: [173.0],
 3766.0: [209.0, 287.0],
 2823.0: [301.0],
 4205.0: [311.0],
 2871.0: [167.0],
 3196.0: [241.0],
 139.0: [266.0],
 2387.0: [189.0],
 801.0: [283.0],
 615.0: [314.0],
 1362.0: [280.0],
 3673.0: [202.0],
 2925.0: [183.0],
 3491.0: [181.0],
 4882.0: [201.0],
 246.0: [281.0],
 4129.0: [312.0],
 1131.0: [190.0],
 419.0: [218.0],
 2085.0: [176.0],
 3003.0: [300.0],
 4138.0: [299.0],
 3939.0: [205.0],
 1598.0: [263.0],
 4224.0: [270.0],
 623.0: [168.0],
 2091.0: [232.0],
 4440.0: [249.0],
 780.0: [208.0],
 1784.0: [226.0],
 524.0: [262.0]

In [19]:
item_count = {}
for key in items.keys():
    item_count[key] = len(items[key])
item_count

{3791.0: 1,
 1.0: 1,
 2781.0: 2,
 3989.0: 1,
 3096.0: 1,
 158.0: 1,
 1765.0: 1,
 4257.0: 1,
 4955.0: 1,
 3828.0: 2,
 3101.0: 1,
 3323.0: 1,
 3083.0: 1,
 1913.0: 1,
 145.0: 2,
 4798.0: 1,
 834.0: 1,
 1641.0: 1,
 4282.0: 1,
 831.0: 1,
 637.0: 1,
 2623.0: 1,
 4760.0: 1,
 2642.0: 1,
 4321.0: 1,
 3766.0: 2,
 2823.0: 1,
 4205.0: 1,
 2871.0: 1,
 3196.0: 1,
 139.0: 1,
 2387.0: 1,
 801.0: 1,
 615.0: 1,
 1362.0: 1,
 3673.0: 1,
 2925.0: 1,
 3491.0: 1,
 4882.0: 1,
 246.0: 1,
 4129.0: 1,
 1131.0: 1,
 419.0: 1,
 2085.0: 1,
 3003.0: 1,
 4138.0: 1,
 3939.0: 1,
 1598.0: 1,
 4224.0: 1,
 623.0: 1,
 2091.0: 1,
 4440.0: 1,
 780.0: 1,
 1784.0: 1,
 524.0: 1,
 3119.0: 1,
 2292.0: 1,
 2159.0: 1,
 4744.0: 1,
 582.0: 1,
 4964.0: 1,
 1087.0: 1,
 1985.0: 1,
 4218.0: 2,
 2560.0: 1,
 4549.0: 1,
 3751.0: 1,
 3769.0: 1,
 4597.0: 1,
 3246.0: 2,
 2960.0: 1,
 4538.0: 1,
 644.0: 1,
 5.0: 1,
 461.0: 1,
 1943.0: 1,
 314.0: 1,
 630.0: 1,
 2277.0: 1,
 1210.0: 1,
 3923.0: 1,
 4185.0: 1,
 4070.0: 1,
 1750.0: 1,
 2820.0: 1,
 332

In [20]:
total_item_count = sum(item_count.values())
total_item_count

155

In [21]:
item_count_diversity = {}
for key in item_count:
    item_count_diversity[key]= item_count[key]/total_item_count
item_count_diversity

{3791.0: 0.0064516129032258064,
 1.0: 0.0064516129032258064,
 2781.0: 0.012903225806451613,
 3989.0: 0.0064516129032258064,
 3096.0: 0.0064516129032258064,
 158.0: 0.0064516129032258064,
 1765.0: 0.0064516129032258064,
 4257.0: 0.0064516129032258064,
 4955.0: 0.0064516129032258064,
 3828.0: 0.012903225806451613,
 3101.0: 0.0064516129032258064,
 3323.0: 0.0064516129032258064,
 3083.0: 0.0064516129032258064,
 1913.0: 0.0064516129032258064,
 145.0: 0.012903225806451613,
 4798.0: 0.0064516129032258064,
 834.0: 0.0064516129032258064,
 1641.0: 0.0064516129032258064,
 4282.0: 0.0064516129032258064,
 831.0: 0.0064516129032258064,
 637.0: 0.0064516129032258064,
 2623.0: 0.0064516129032258064,
 4760.0: 0.0064516129032258064,
 2642.0: 0.0064516129032258064,
 4321.0: 0.0064516129032258064,
 3766.0: 0.012903225806451613,
 2823.0: 0.0064516129032258064,
 4205.0: 0.0064516129032258064,
 2871.0: 0.0064516129032258064,
 3196.0: 0.0064516129032258064,
 139.0: 0.0064516129032258064,
 2387.0: 0.0064516129

In [22]:
df_merchant['merchant product diversity'] = -1
for index,row in df_merchant.iterrows():
        df_merchant.at[index, 'merchant product diversity'] = item_count_diversity[row['seller_id']]

df_merchant

/tmp/ipykernel_17092/1602293768.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merchant['merchant product diversity'] = -1


,seller_id,merchant action count,merchant action count ratio,merchant day count,merchant product diversity
0,3791,14,0.002139,11,0.006452
1,3791,14,0.002139,11,0.006452
2,1,45,0.006874,36,0.006452
3,1,45,0.006874,36,0.006452
4,1,45,0.006874,36,0.006452
...,...,...,...,...,...
6544,637,885,0.135197,149,0.006452
6545,4760,398,0.060800,36,0.006452
6546,1200,13,0.001986,4,0.006452
6547,1200,13,0.001986,4,0.006452


In [23]:
#FEATURE 9: DOUBLE 11 FEATURES
df_11 = df[df['time_stamp']==1111]
df_11

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,age_range,gender,prob
0,149002,282,885,3791,1500.0,1111,0,5.0,2.0,NaN
5,142526,193,656,2781,6885.0,1111,0,6.0,0.0,NaN
11,46520,257,1238,158,99.0,1111,0,4.0,0.0,NaN
22,14747,279,898,3323,683.0,1111,0,6.0,1.0,NaN
31,379824,198,656,145,3462.0,1111,0,5.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...
6529,107147,259,614,637,2603.0,1111,0,0.0,1.0,NaN
6538,127336,259,614,637,2603.0,1111,0,0.0,0.0,NaN
6544,73105,259,614,637,2603.0,1111,0,2.0,0.0,NaN
6545,403421,175,1181,4760,247.0,1111,0,3.0,1.0,NaN


In [24]:
actions_11 = {}

# go through user_seller sub-dataframe and split entries by user_id:seller_id
for index, row in df_11.iterrows():
    if row['seller_id'] not in actions_11:
        # there may be multiple entries, so default with an empty list
        actions_11[row["seller_id"]] = [] 
    actions_11[row["seller_id"]].append(row["action_type"])

In [25]:
#for k,v in actions_11.items():
#    actions_11[k] = list(set(v)) # make each value list unique

In [26]:
actions_11

{3791.0: [0.0],
 2781.0: [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  2.0,
  3.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  2.0,
  0.0,
  2.0,
  0.0,
  0.0,
  0.0],
 158.0: [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  2.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 3323.0: [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,


In [27]:
action_count_11 = {}
for key in actions_11.keys():
    action_count_11[key] = len(actions_11[key])
action_count_11

{3791.0: 1,
 2781.0: 69,
 158.0: 17,
 3323.0: 163,
 145.0: 66,
 834.0: 9,
 1641.0: 62,
 831.0: 5,
 4760.0: 234,
 4798.0: 96,
 4205.0: 4,
 1362.0: 2,
 3989.0: 22,
 246.0: 11,
 637.0: 102,
 1131.0: 11,
 2387.0: 3,
 3766.0: 19,
 1784.0: 17,
 2085.0: 11,
 4955.0: 49,
 4218.0: 19,
 2560.0: 6,
 1943.0: 21,
 2277.0: 39,
 3096.0: 11,
 524.0: 2,
 1210.0: 2,
 4185.0: 4,
 1087.0: 26,
 586.0: 2,
 1378.0: 1,
 3263.0: 15,
 1200.0: 4,
 259.0: 2,
 314.0: 21,
 3246.0: 1,
 3593.0: 2,
 1294.0: 2,
 1536.0: 2,
 1598.0: 9,
 4561.0: 2,
 615.0: 3,
 1606.0: 13,
 3119.0: 1,
 4405.0: 5,
 1765.0: 4,
 419.0: 1,
 2340.0: 1,
 3101.0: 8,
 630.0: 1,
 2159.0: 1,
 2822.0: 5,
 2846.0: 2,
 3003.0: 1,
 2608.0: 1,
 2642.0: 1,
 3209.0: 2,
 3083.0: 1}

In [28]:
total_action_count_11 = sum(action_count_11.values())
total_action_count_11

1217

In [29]:
action_count_ratio_11 = {}
for key in action_count_11:
    action_count_ratio_11[key] = action_count_11[key]/total_action_count_11
action_count_ratio_11

{3791.0: 0.0008216926869350862,
 2781.0: 0.056696795398520954,
 158.0: 0.013968775677896467,
 3323.0: 0.13393590797041907,
 145.0: 0.054231717337715694,
 834.0: 0.007395234182415777,
 1641.0: 0.05094494658997535,
 831.0: 0.004108463434675432,
 4760.0: 0.1922760887428102,
 4798.0: 0.07888249794576828,
 4205.0: 0.003286770747740345,
 1362.0: 0.0016433853738701725,
 3989.0: 0.018077239112571898,
 246.0: 0.009038619556285949,
 637.0: 0.0838126540673788,
 1131.0: 0.009038619556285949,
 2387.0: 0.0024650780608052587,
 3766.0: 0.015612161051766639,
 1784.0: 0.013968775677896467,
 2085.0: 0.009038619556285949,
 4955.0: 0.04026294165981923,
 4218.0: 0.015612161051766639,
 2560.0: 0.0049301561216105174,
 1943.0: 0.01725554642563681,
 2277.0: 0.03204601479046836,
 3096.0: 0.009038619556285949,
 524.0: 0.0016433853738701725,
 1210.0: 0.0016433853738701725,
 4185.0: 0.003286770747740345,
 1087.0: 0.021364009860312245,
 586.0: 0.0016433853738701725,
 1378.0: 0.0008216926869350862,
 3263.0: 0.0123253

In [30]:
df_merchant['merchant 1111 action count'] = 0
df_merchant['merchant 1111 action count ratio'] = 0
for index,row in df_merchant.iterrows():
        if row['seller_id'] in action_count_11.keys():
            df_merchant.at[index, 'merchant 1111 action count'] = action_count_11[row['seller_id']]
        if row['seller_id'] in action_count_ratio_11.keys():
            df_merchant.at[index, 'merchant 1111 action count ratio'] = action_count_ratio_11[row['seller_id']]

df_merchant

/tmp/ipykernel_17092/3269018672.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merchant['merchant 1111 action count'] = 0
/tmp/ipykernel_17092/3269018672.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merchant['merchant 1111 action count ratio'] = 0


,seller_id,merchant action count,merchant action count ratio,merchant day count,merchant product diversity,merchant 1111 action count,merchant 1111 action count ratio
0,3791,14,0.002139,11,0.006452,1,0.000822
1,3791,14,0.002139,11,0.006452,1,0.000822
2,1,45,0.006874,36,0.006452,0,0.000000
3,1,45,0.006874,36,0.006452,0,0.000000
4,1,45,0.006874,36,0.006452,0,0.000000
...,...,...,...,...,...,...,...
6544,637,885,0.135197,149,0.006452,102,0.083813
6545,4760,398,0.060800,36,0.006452,234,0.192276
6546,1200,13,0.001986,4,0.006452,4,0.003287
6547,1200,13,0.001986,4,0.006452,4,0.003287


In [31]:
activity_ratio_1111 = {}
df_merchant['merchant 1111 activity ratio'] = 0
for index,row in df_merchant.iterrows():
    if df_merchant.at[index,'merchant 1111 action count'] != 0:
        df_merchant.at[index, 'merchant 1111 activity ratio'] = (df_merchant.at[index,'merchant 1111 action count']/df_merchant.at[index,'merchant action count'])
df_merchant

/tmp/ipykernel_17092/817184100.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merchant['merchant 1111 activity ratio'] = 0


,seller_id,merchant action count,merchant action count ratio,merchant day count,merchant product diversity,merchant 1111 action count,merchant 1111 action count ratio,merchant 1111 activity ratio
0,3791,14,0.002139,11,0.006452,1,0.000822,0.071429
1,3791,14,0.002139,11,0.006452,1,0.000822,0.071429
2,1,45,0.006874,36,0.006452,0,0.000000,0.000000
3,1,45,0.006874,36,0.006452,0,0.000000,0.000000
4,1,45,0.006874,36,0.006452,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...
6544,637,885,0.135197,149,0.006452,102,0.083813,0.115254
6545,4760,398,0.060800,36,0.006452,234,0.192276,0.587940
6546,1200,13,0.001986,4,0.006452,4,0.003287,0.307692
6547,1200,13,0.001986,4,0.006452,4,0.003287,0.307692


In [32]:
# LATEST ONE WEEK
latest_week = 1104
df_latest_week = df[df['time_stamp']>=latest_week]
df_latest_week

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,age_range,gender,prob
0,149002,282,885,3791,1500.0,1111,0,5.0,2.0,NaN
1,149002,282,885,3791,1500.0,1107,0,5.0,2.0,NaN
5,142526,193,656,2781,6885.0,1111,0,6.0,0.0,NaN
11,46520,257,1238,158,99.0,1111,0,4.0,0.0,NaN
22,14747,279,898,3323,683.0,1111,0,6.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...
6538,127336,259,614,637,2603.0,1111,0,0.0,0.0,NaN
6544,73105,259,614,637,2603.0,1111,0,2.0,0.0,NaN
6545,403421,175,1181,4760,247.0,1111,0,3.0,1.0,NaN
6546,116738,186,267,1200,2276.0,1110,0,2.0,1.0,NaN


In [33]:
actions_last_week = {}

# go through user_seller sub-dataframe and split entries by seller_id:seller_id
for index, row in df_latest_week.iterrows():
    if row['seller_id'] not in actions_last_week:
        # there may be multiple entries, so default with an empty list
        actions_last_week[row["seller_id"]] = [] 
    actions_last_week[row["seller_id"]].append(row["action_type"])
    
#for k,v in actions_last_week.items():
#    actions_last_week[k] = list(set(v)) # make each value list unique
actions_last_week

{3791.0: [0.0, 0.0, 0.0],
 2781.0: [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  2.0,
  3.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  2.0,
  0.0,
  0.0,
  2.0,
  0.0,
  0.0,
  0.0,
  3.0,
  0.0],
 158.0: [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  2.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 3323.0: [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.

In [34]:
action_count_last_week = {}
for key in actions_last_week.keys():
    action_count_last_week[key] = len(actions_last_week[key])
action_count_last_week

{3791.0: 3,
 2781.0: 90,
 158.0: 21,
 3323.0: 289,
 1913.0: 3,
 145.0: 101,
 834.0: 31,
 1641.0: 135,
 831.0: 5,
 4760.0: 330,
 2642.0: 2,
 4798.0: 174,
 4205.0: 5,
 2387.0: 61,
 615.0: 7,
 1362.0: 2,
 3989.0: 71,
 246.0: 18,
 637.0: 190,
 1131.0: 42,
 1598.0: 12,
 4224.0: 1,
 3766.0: 36,
 1784.0: 29,
 2085.0: 17,
 4955.0: 51,
 1087.0: 35,
 4218.0: 27,
 2560.0: 12,
 1943.0: 25,
 2277.0: 69,
 3096.0: 13,
 524.0: 2,
 1210.0: 2,
 4185.0: 9,
 4138.0: 1,
 586.0: 2,
 1378.0: 1,
 3263.0: 23,
 3083.0: 4,
 2732.0: 3,
 2751.0: 2,
 630.0: 4,
 1200.0: 12,
 259.0: 5,
 2340.0: 5,
 4405.0: 16,
 314.0: 24,
 3246.0: 1,
 1750.0: 1,
 3593.0: 2,
 1294.0: 2,
 3911.0: 2,
 1536.0: 2,
 1765.0: 6,
 2522.0: 1,
 2822.0: 9,
 4561.0: 4,
 1606.0: 17,
 3209.0: 5,
 3119.0: 1,
 419.0: 1,
 4988.0: 2,
 3101.0: 9,
 2159.0: 2,
 3196.0: 2,
 2443.0: 1,
 4129.0: 4,
 2846.0: 6,
 3003.0: 1,
 2608.0: 1,
 2226.0: 1}

In [35]:
total_action_count_last_week = sum(action_count_last_week.values())
total_action_count_last_week

2105

In [36]:
action_count_ratio_last_week = {}
for key in action_count_last_week:
    action_count_ratio_last_week[key] = action_count_last_week[key]/total_action_count_last_week
action_count_ratio_last_week

{3791.0: 0.0014251781472684087,
 2781.0: 0.04275534441805225,
 158.0: 0.009976247030878859,
 3323.0: 0.13729216152019003,
 1913.0: 0.0014251781472684087,
 145.0: 0.047980997624703085,
 834.0: 0.014726840855106888,
 1641.0: 0.06413301662707839,
 831.0: 0.0023752969121140144,
 4760.0: 0.15676959619952494,
 2642.0: 0.0009501187648456057,
 4798.0: 0.0826603325415677,
 4205.0: 0.0023752969121140144,
 2387.0: 0.028978622327790974,
 615.0: 0.00332541567695962,
 1362.0: 0.0009501187648456057,
 3989.0: 0.033729216152019,
 246.0: 0.00855106888361045,
 637.0: 0.09026128266033254,
 1131.0: 0.019952494061757718,
 1598.0: 0.005700712589073635,
 4224.0: 0.00047505938242280285,
 3766.0: 0.0171021377672209,
 1784.0: 0.013776722090261283,
 2085.0: 0.008076009501187649,
 4955.0: 0.024228028503562947,
 1087.0: 0.0166270783847981,
 4218.0: 0.012826603325415678,
 2560.0: 0.005700712589073635,
 1943.0: 0.011876484560570071,
 2277.0: 0.0327790973871734,
 3096.0: 0.006175771971496437,
 524.0: 0.000950118764845

In [37]:
df_merchant['merchant last week action count'] = 0
df_merchant['merchant last week action count ratio'] = 0
for index,row in df_merchant.iterrows():
        if row['seller_id'] in action_count_last_week.keys():
            df_merchant.at[index, 'merchant last week action count'] = action_count_last_week[row['seller_id']]
        if row['seller_id'] in action_count_ratio_last_week.keys():
            df_merchant.at[index, 'merchant last week action count ratio'] = action_count_ratio_last_week[row['seller_id']]

df_merchant

/tmp/ipykernel_17092/11014329.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merchant['merchant last week action count'] = 0


,seller_id,merchant action count,merchant action count ratio,merchant day count,merchant product diversity,merchant 1111 action count,merchant 1111 action count ratio,merchant 1111 activity ratio,merchant last week action count,merchant last week action count ratio
0,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425
1,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425
2,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000
3,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000
4,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000
...,...,...,...,...,...,...,...,...,...,...
6544,637,885,0.135197,149,0.006452,102,0.083813,0.115254,190,0.090261
6545,4760,398,0.060800,36,0.006452,234,0.192276,0.587940,330,0.156770
6546,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701
6547,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701


In [38]:
activity_ratio_last_week = {}
df_merchant['merchant last week activity ratio'] = 0
for index,row in df_merchant.iterrows():
    if df_merchant.at[index,'merchant last week action count'] != 0:
        df_merchant.at[index, 'merchant last week activity ratio'] = (df_merchant.at[index,'merchant last week action count']/df_merchant.at[index,'merchant action count'])
df_merchant

,seller_id,merchant action count,merchant action count ratio,merchant day count,merchant product diversity,merchant 1111 action count,merchant 1111 action count ratio,merchant 1111 activity ratio,merchant last week action count,merchant last week action count ratio,merchant last week activity ratio
0,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,0.214286
1,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,0.214286
2,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,0.000000
3,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,0.000000
4,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
6544,637,885,0.135197,149,0.006452,102,0.083813,0.115254,190,0.090261,0.214689
6545,4760,398,0.060800,36,0.006452,234,0.192276,0.587940,330,0.156770,0.829146
6546,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,0.923077
6547,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,0.923077


In [39]:
#FEATURE 11: LATEST MONTH
latest_month = 1011
df_latest_month = df[df['time_stamp']>=latest_month]
df_latest_month

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,age_range,gender,prob
0,149002,282,885,3791,1500.0,1111,0,5.0,2.0,NaN
1,149002,282,885,3791,1500.0,1107,0,5.0,2.0,NaN
5,142526,193,656,2781,6885.0,1111,0,6.0,0.0,NaN
10,359170,217,1326,3096,3080.0,1021,3,3.0,0.0,NaN
11,46520,257,1238,158,99.0,1111,0,4.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...
6544,73105,259,614,637,2603.0,1111,0,2.0,0.0,NaN
6545,403421,175,1181,4760,247.0,1111,0,3.0,1.0,NaN
6546,116738,186,267,1200,2276.0,1110,0,2.0,1.0,NaN
6547,116738,186,267,1200,2276.0,1101,0,2.0,1.0,NaN


In [40]:
actions_last_month = {}

# go through user_seller sub-dataframe and split entries by user_id:seller_id
for index, row in df_latest_month.iterrows():
    if row['seller_id'] not in actions_last_month:
        # there may be multiple entries, so default with an empty list
        actions_last_month[row["seller_id"]] = [] 
    actions_last_month[row["seller_id"]].append(row["action_type"])
    
#for k,v in actions_last_month.items():
#    actions_last_month[k] = list(set(v)) # make each value list unique
actions_last_month

{3791.0: [0.0, 0.0, 0.0, 0.0],
 2781.0: [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  2.0,
  3.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  2.0,
  0.0,
  0.0,
  2.0,
  0.0,
  0.0,
  0.0,
  3.0,
  0.0],
 3096.0: [3.0,
  0.0,
  0.0,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 158.0: [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  2.0,
  0.0,
  0.0,
  0.0,
  0.0,

In [41]:
action_count_last_month = {}
for key in actions_last_month.keys():
    action_count_last_month[key] = len(actions_last_month[key])
action_count_last_month

{3791.0: 4,
 2781.0: 94,
 3096.0: 27,
 158.0: 23,
 3989.0: 155,
 3323.0: 297,
 4257.0: 8,
 3083.0: 26,
 1913.0: 4,
 145.0: 111,
 4798.0: 227,
 834.0: 31,
 1641.0: 165,
 831.0: 6,
 4760.0: 390,
 2642.0: 2,
 4205.0: 9,
 3196.0: 35,
 2387.0: 61,
 615.0: 12,
 3101.0: 11,
 1362.0: 2,
 246.0: 20,
 637.0: 376,
 1131.0: 42,
 1598.0: 18,
 4224.0: 1,
 1784.0: 30,
 3119.0: 3,
 3766.0: 43,
 4129.0: 10,
 2085.0: 23,
 4955.0: 51,
 4964.0: 1,
 1087.0: 36,
 4218.0: 28,
 2560.0: 12,
 2823.0: 1,
 4597.0: 4,
 1943.0: 25,
 2277.0: 72,
 524.0: 2,
 1210.0: 4,
 4138.0: 20,
 4185.0: 15,
 1750.0: 2,
 1.0: 1,
 586.0: 2,
 1378.0: 1,
 2751.0: 9,
 3263.0: 23,
 4321.0: 3,
 4405.0: 21,
 2732.0: 3,
 4988.0: 4,
 801.0: 1,
 630.0: 4,
 1200.0: 13,
 259.0: 5,
 2340.0: 6,
 2226.0: 5,
 314.0: 25,
 3246.0: 1,
 3593.0: 2,
 419.0: 4,
 1294.0: 2,
 4561.0: 5,
 1536.0: 5,
 3911.0: 3,
 1765.0: 8,
 2522.0: 1,
 2822.0: 9,
 3055.0: 5,
 1606.0: 17,
 2820.0: 1,
 4267.0: 1,
 3209.0: 5,
 2846.0: 8,
 4573.0: 1,
 2159.0: 2,
 1347.0: 2,
 2

In [42]:
total_action_count_last_month = sum(action_count_last_month.values())
total_action_count_last_month

2752

In [43]:
action_count_ratio_last_month = {}
for key in action_count_last_month:
    action_count_ratio_last_month[key] = action_count_last_month[key]/total_action_count_last_month
action_count_ratio_last_month

{3791.0: 0.0014534883720930232,
 2781.0: 0.034156976744186045,
 3096.0: 0.009811046511627907,
 158.0: 0.008357558139534883,
 3989.0: 0.056322674418604654,
 3323.0: 0.10792151162790697,
 4257.0: 0.0029069767441860465,
 3083.0: 0.00944767441860465,
 1913.0: 0.0014534883720930232,
 145.0: 0.0403343023255814,
 4798.0: 0.08248546511627906,
 834.0: 0.01126453488372093,
 1641.0: 0.05995639534883721,
 831.0: 0.002180232558139535,
 4760.0: 0.14171511627906977,
 2642.0: 0.0007267441860465116,
 4205.0: 0.0032703488372093025,
 3196.0: 0.012718023255813954,
 2387.0: 0.022165697674418606,
 615.0: 0.00436046511627907,
 3101.0: 0.003997093023255814,
 1362.0: 0.0007267441860465116,
 246.0: 0.007267441860465116,
 637.0: 0.13662790697674418,
 1131.0: 0.015261627906976744,
 1598.0: 0.006540697674418605,
 4224.0: 0.0003633720930232558,
 1784.0: 0.010901162790697675,
 3119.0: 0.0010901162790697674,
 3766.0: 0.015625,
 4129.0: 0.003633720930232558,
 2085.0: 0.008357558139534883,
 4955.0: 0.018531976744186048

In [44]:
df_merchant['last month action count'] = 0
df_merchant['last month action count ratio'] = 0
for index,row in df_merchant.iterrows():
        if row['seller_id'] in action_count_last_month.keys():
            df_merchant.at[index, 'last month action count'] = action_count_last_month[row['seller_id']]
        if row['seller_id'] in action_count_ratio_last_month.keys():
            df_merchant.at[index, 'last month action count ratio'] = action_count_ratio_last_month[row['seller_id']]

df_merchant

,seller_id,merchant action count,merchant action count ratio,merchant day count,merchant product diversity,merchant 1111 action count,merchant 1111 action count ratio,merchant 1111 activity ratio,merchant last week action count,merchant last week action count ratio,merchant last week activity ratio,last month action count,last month action count ratio
0,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,0.214286,4,0.001453
1,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,0.214286,4,0.001453
2,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,0.000000,1,0.000363
3,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,0.000000,1,0.000363
4,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,0.000000,1,0.000363
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6544,637,885,0.135197,149,0.006452,102,0.083813,0.115254,190,0.090261,0.214689,376,0.136628
6545,4760,398,0.060800,36,0.006452,234,0.192276,0.587940,330,0.156770,0.829146,390,0.141715
6546,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,0.923077,13,0.004724
6547,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,0.923077,13,0.004724


In [45]:
activity_ratio_last_month = {}
df_merchant['last month activity ratio'] = 0
for index,row in df_merchant.iterrows():
    if df_merchant.at[index,'last month action count'] != 0:
        df_merchant.at[index, 'last month activity ratio'] = (df_merchant.at[index,'last month action count']/df_merchant.at[index,'merchant action count'])
df_merchant

,seller_id,merchant action count,merchant action count ratio,merchant day count,merchant product diversity,merchant 1111 action count,merchant 1111 action count ratio,merchant 1111 activity ratio,merchant last week action count,merchant last week action count ratio,merchant last week activity ratio,last month action count,last month action count ratio,last month activity ratio
0,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,0.214286,4,0.001453,0.285714
1,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,0.214286,4,0.001453,0.285714
2,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,0.000000,1,0.000363,0.022222
3,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,0.000000,1,0.000363,0.022222
4,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,0.000000,1,0.000363,0.022222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6544,637,885,0.135197,149,0.006452,102,0.083813,0.115254,190,0.090261,0.214689,376,0.136628,0.424859
6545,4760,398,0.060800,36,0.006452,234,0.192276,0.587940,330,0.156770,0.829146,390,0.141715,0.979899
6546,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,0.923077,13,0.004724,1.000000
6547,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,0.923077,13,0.004724,1.000000


In [46]:
print("TOTAL ACTIONS: ", total_action_count)
print("TOTAL ACTIONS LATEST MONTH: ",total_action_count_last_month)
print("TOTAL ACTIONS LATEST WEEK: ",total_action_count_last_week)
print("TOTAL ACTIONS 1111: ",total_action_count_11)

TOTAL ACTIONS:  6546
TOTAL ACTIONS LATEST MONTH:  2752
TOTAL ACTIONS LATEST WEEK:  2105
TOTAL ACTIONS 1111:  1217


In [47]:
#FEATURE 18: AGE RELATED FEATURES
merchant_age = {}

# go through user_seller sub-dataframe and split entries by user_id:seller_id
for index, row in df.iterrows():
    if row['seller_id'] not in merchant_age:
        # there may be multiple entries, so default with an empty list
        merchant_age[row["seller_id"]] = [] 
    merchant_age[row["seller_id"]].append(row["age_range"])

merchant_age_avg = {}
for row in merchant_age.items():
    merchant_age_avg[row[0]] = np.nansum(row[1])/len(row[1])
sorted(merchant_age_avg.items())

[(1.0, 3.4),
 (5.0, 2.5),
 (25.0, 4.25),
 (107.0, 3.0),
 (139.0, 3.7),
 (145.0, 3.4864864864864864),
 (158.0, 3.5652173913043477),
 (246.0, 3.95),
 (259.0, 2.2),
 (314.0, 1.5185185185185186),
 (388.0, 3.0),
 (419.0, 3.356164383561644),
 (448.0, 6.0),
 (461.0, 1.7142857142857142),
 (503.0, 4.0),
 (524.0, 2.857142857142857),
 (582.0, 2.75),
 (586.0, 4.5),
 (615.0, 2.3333333333333335),
 (623.0, 3.5555555555555554),
 (630.0, 3.3043478260869565),
 (637.0, 2.47909604519774),
 (644.0, 2.8),
 (707.0, 2.0),
 (780.0, 3.1538461538461537),
 (801.0, 2.125),
 (831.0, 3.4285714285714284),
 (834.0, 3.2580645161290325),
 (1087.0, 3.0),
 (1102.0, 2.0),
 (1131.0, 3.4047619047619047),
 (1200.0, 2.1538461538461537),
 (1210.0, 2.5),
 (1294.0, 3.1818181818181817),
 (1347.0, 2.0),
 (1362.0, 3.5),
 (1368.0, 0.9),
 (1378.0, 4.0),
 (1473.0, 3.0),
 (1536.0, 3.5714285714285716),
 (1598.0, 3.1),
 (1606.0, 5.944444444444445),
 (1641.0, 3.425373134328358),
 (1734.0, 4.538461538461538),
 (1746.0, 3.5),
 (1750.0, 0.0),

In [48]:
total_merchant_average_age = sum(merchant_age_avg.values())/len(merchant_age_avg.keys())
print("Average age total: ",total_merchant_average_age)

Average age total:  3.1021988412029224


In [49]:
df_merchant['average age'] = 0
for index,row in df_merchant.iterrows():
        if row['seller_id'] in merchant_age_avg.keys():
            df_merchant.at[index, 'average age'] = merchant_age_avg[row['seller_id']]
df_merchant

,seller_id,merchant action count,merchant action count ratio,merchant day count,merchant product diversity,merchant 1111 action count,merchant 1111 action count ratio,merchant 1111 activity ratio,merchant last week action count,merchant last week action count ratio,merchant last week activity ratio,last month action count,last month action count ratio,last month activity ratio,average age
0,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,0.214286,4,0.001453,0.285714,4.500000
1,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,0.214286,4,0.001453,0.285714,4.500000
2,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,0.000000,1,0.000363,0.022222,3.400000
3,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,0.000000,1,0.000363,0.022222,3.400000
4,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,0.000000,1,0.000363,0.022222,3.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6544,637,885,0.135197,149,0.006452,102,0.083813,0.115254,190,0.090261,0.214689,376,0.136628,0.424859,2.479096
6545,4760,398,0.060800,36,0.006452,234,0.192276,0.587940,330,0.156770,0.829146,390,0.141715,0.979899,2.479899
6546,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,0.923077,13,0.004724,1.000000,2.153846
6547,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,0.923077,13,0.004724,1.000000,2.153846


In [50]:
#FEATURE 19: GENDER FEATURES
merchant_gender = {}

# go through user_seller sub-dataframe and split entries by user_id:seller_id
for index, row in df.iterrows():
    if row['seller_id'] not in merchant_gender:
        # there may be multiple entries, so default with an empty list
        merchant_gender[row["seller_id"]] = [] 
    merchant_gender[row["seller_id"]].append(row["gender"])

merchant_gender_avg = {}
for row in merchant_gender.items():
    merchant_gender_avg[row[0]] = np.nansum(row[1])/len(row[1])
sorted(merchant_gender_avg.items())

[(1.0, 0.26666666666666666),
 (5.0, 0.5),
 (25.0, 0.5),
 (107.0, 0.0),
 (139.0, 0.0),
 (145.0, 0.2072072072072072),
 (158.0, 0.2608695652173913),
 (246.0, 0.05),
 (259.0, 1.0),
 (314.0, 0.7777777777777778),
 (388.0, 2.0),
 (419.0, 0.6301369863013698),
 (448.0, 1.0),
 (461.0, 0.0),
 (503.0, 0.0),
 (524.0, 0.14285714285714285),
 (582.0, 0.5),
 (586.0, 0.5),
 (615.0, 0.3333333333333333),
 (623.0, 0.4444444444444444),
 (630.0, 0.2608695652173913),
 (637.0, 0.25084745762711863),
 (644.0, 0.075),
 (707.0, 0.5),
 (780.0, 0.0),
 (801.0, 0.05555555555555555),
 (831.0, 0.07142857142857142),
 (834.0, 0.45161290322580644),
 (1087.0, 0.3888888888888889),
 (1102.0, 0.0),
 (1131.0, 0.09523809523809523),
 (1200.0, 0.15384615384615385),
 (1210.0, 1.0),
 (1294.0, 1.0909090909090908),
 (1347.0, 0.0),
 (1362.0, 0.0),
 (1368.0, 0.1),
 (1378.0, 1.0),
 (1473.0, 1.0),
 (1536.0, 0.0),
 (1598.0, 0.25),
 (1606.0, 1.7777777777777777),
 (1641.0, 0.3880597014925373),
 (1734.0, 0.0),
 (1746.0, 0.3333333333333333),
 

In [51]:
total_merchant_average_gender = sum(merchant_gender_avg.values())/len(merchant_gender_avg.keys())
print("Average gender total: ",total_merchant_average_gender)

Average gender total:  0.34803445560052887


In [52]:
df_merchant['average gender'] = 0
for index,row in df_merchant.iterrows():
        if row['seller_id'] in merchant_gender_avg.keys():
            df_merchant.at[index, 'average gender'] = merchant_gender_avg[row['seller_id']]
df_merchant

,seller_id,merchant action count,merchant action count ratio,merchant day count,merchant product diversity,merchant 1111 action count,merchant 1111 action count ratio,merchant 1111 activity ratio,merchant last week action count,merchant last week action count ratio,merchant last week activity ratio,last month action count,last month action count ratio,last month activity ratio,average age,average gender
0,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,0.214286,4,0.001453,0.285714,4.500000,0.357143
1,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,0.214286,4,0.001453,0.285714,4.500000,0.357143
2,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,0.000000,1,0.000363,0.022222,3.400000,0.266667
3,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,0.000000,1,0.000363,0.022222,3.400000,0.266667
4,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,0.000000,1,0.000363,0.022222,3.400000,0.266667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6544,637,885,0.135197,149,0.006452,102,0.083813,0.115254,190,0.090261,0.214689,376,0.136628,0.424859,2.479096,0.250847
6545,4760,398,0.060800,36,0.006452,234,0.192276,0.587940,330,0.156770,0.829146,390,0.141715,0.979899,2.479899,0.590452
6546,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,0.923077,13,0.004724,1.000000,2.153846,0.153846
6547,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,0.923077,13,0.004724,1.000000,2.153846,0.153846


# Merging Features (James)

In [53]:
df['month'] = df['time_stamp'] // 100
df['day'] = df['time_stamp'] % 100
df

/tmp/ipykernel_17092/3691477280.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['month'] = df['time_stamp'] // 100
/tmp/ipykernel_17092/3691477280.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['day'] = df['time_stamp'] % 100


,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,age_range,gender,prob,month,day
0,149002,282,885,3791,1500.0,1111,0,5.0,2.0,NaN,11,11
1,149002,282,885,3791,1500.0,1107,0,5.0,2.0,NaN,11,7
2,109881,230,629,1,1662.0,820,0,4.0,0.0,NaN,8,20
3,109881,230,629,1,1662.0,820,0,4.0,0.0,NaN,8,20
4,109881,230,629,1,1662.0,820,0,4.0,0.0,NaN,8,20
...,...,...,...,...,...,...,...,...,...,...,...,...
6544,73105,259,614,637,2603.0,1111,0,2.0,0.0,NaN,11,11
6545,403421,175,1181,4760,247.0,1111,0,3.0,1.0,NaN,11,11
6546,116738,186,267,1200,2276.0,1110,0,2.0,1.0,NaN,11,10
6547,116738,186,267,1200,2276.0,1101,0,2.0,1.0,NaN,11,1


In [54]:
# Split Month Day into Separate Columns
df_merchant['month'] = df['time_stamp'] // 100
df_merchant['day'] = df['time_stamp'] % 100
df_merchant

,seller_id,merchant action count,merchant action count ratio,merchant day count,merchant product diversity,merchant 1111 action count,merchant 1111 action count ratio,merchant 1111 activity ratio,merchant last week action count,merchant last week action count ratio,merchant last week activity ratio,last month action count,last month action count ratio,last month activity ratio,average age,average gender,month,day
0,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,0.214286,4,0.001453,0.285714,4.500000,0.357143,11,11
1,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,0.214286,4,0.001453,0.285714,4.500000,0.357143,11,7
2,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,0.000000,1,0.000363,0.022222,3.400000,0.266667,8,20
3,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,0.000000,1,0.000363,0.022222,3.400000,0.266667,8,20
4,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,0.000000,1,0.000363,0.022222,3.400000,0.266667,8,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6544,637,885,0.135197,149,0.006452,102,0.083813,0.115254,190,0.090261,0.214689,376,0.136628,0.424859,2.479096,0.250847,11,11
6545,4760,398,0.060800,36,0.006452,234,0.192276,0.587940,330,0.156770,0.829146,390,0.141715,0.979899,2.479899,0.590452,11,11
6546,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,0.923077,13,0.004724,1.000000,2.153846,0.153846,11,10
6547,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,0.923077,13,0.004724,1.000000,2.153846,0.153846,11,1


In [55]:
# Monthly Merchant Action Counts
df_merchant['merchant_monthly_action_count'] = df_merchant.groupby(['seller_id', 'month']).transform('size')
df_merchant

,seller_id,merchant action count,merchant action count ratio,merchant day count,merchant product diversity,merchant 1111 action count,merchant 1111 action count ratio,merchant 1111 activity ratio,merchant last week action count,merchant last week action count ratio,merchant last week activity ratio,last month action count,last month action count ratio,last month activity ratio,average age,average gender,month,day,merchant_monthly_action_count
0,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,0.214286,4,0.001453,0.285714,4.500000,0.357143,11,11,3
1,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,0.214286,4,0.001453,0.285714,4.500000,0.357143,11,7,3
2,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,0.000000,1,0.000363,0.022222,3.400000,0.266667,8,20,10
3,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,0.000000,1,0.000363,0.022222,3.400000,0.266667,8,20,10
4,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,0.000000,1,0.000363,0.022222,3.400000,0.266667,8,20,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6544,637,885,0.135197,149,0.006452,102,0.083813,0.115254,190,0.090261,0.214689,376,0.136628,0.424859,2.479096,0.250847,11,11,225
6545,4760,398,0.060800,36,0.006452,234,0.192276,0.587940,330,0.156770,0.829146,390,0.141715,0.979899,2.479899,0.590452,11,11,342
6546,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,0.923077,13,0.004724,1.000000,2.153846,0.153846,11,10,13
6547,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,0.923077,13,0.004724,1.000000,2.153846,0.153846,11,1,13


In [56]:
# Total Action Count in said month
df_merchant['month_total_action_count'] = df_merchant.groupby(['month']).transform('size')
df_merchant

,seller_id,merchant action count,merchant action count ratio,merchant day count,merchant product diversity,merchant 1111 action count,merchant 1111 action count ratio,merchant 1111 activity ratio,merchant last week action count,merchant last week action count ratio,merchant last week activity ratio,last month action count,last month action count ratio,last month activity ratio,average age,average gender,month,day,merchant_monthly_action_count,month_total_action_count
0,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,0.214286,4,0.001453,0.285714,4.500000,0.357143,11,11,3,2291
1,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,0.214286,4,0.001453,0.285714,4.500000,0.357143,11,7,3,2291
2,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,0.000000,1,0.000363,0.022222,3.400000,0.266667,8,20,10,704
3,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,0.000000,1,0.000363,0.022222,3.400000,0.266667,8,20,10,704
4,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,0.000000,1,0.000363,0.022222,3.400000,0.266667,8,20,10,704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6544,637,885,0.135197,149,0.006452,102,0.083813,0.115254,190,0.090261,0.214689,376,0.136628,0.424859,2.479096,0.250847,11,11,225,2291
6545,4760,398,0.060800,36,0.006452,234,0.192276,0.587940,330,0.156770,0.829146,390,0.141715,0.979899,2.479899,0.590452,11,11,342,2291
6546,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,0.923077,13,0.004724,1.000000,2.153846,0.153846,11,10,13,2291
6547,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,0.923077,13,0.004724,1.000000,2.153846,0.153846,11,1,13,2291


In [57]:
# Monthly Action Count / Ratio (Count/Ratio Type)
df_merchant['monthly_action_count_ratio'] = df_merchant.groupby(['seller_id', 'month'])['merchant_monthly_action_count'].transform(lambda x: x/ df_merchant['month_total_action_count'])
df_merchant



,seller_id,merchant action count,merchant action count ratio,merchant day count,merchant product diversity,merchant 1111 action count,merchant 1111 action count ratio,merchant 1111 activity ratio,merchant last week action count,merchant last week action count ratio,...,last month action count,last month action count ratio,last month activity ratio,average age,average gender,month,day,merchant_monthly_action_count,month_total_action_count,monthly_action_count_ratio
0,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,...,4,0.001453,0.285714,4.500000,0.357143,11,11,3,2291,0.001309
1,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,...,4,0.001453,0.285714,4.500000,0.357143,11,7,3,2291,0.001309
2,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,1,0.000363,0.022222,3.400000,0.266667,8,20,10,704,0.014205
3,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,1,0.000363,0.022222,3.400000,0.266667,8,20,10,704,0.014205
4,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,1,0.000363,0.022222,3.400000,0.266667,8,20,10,704,0.014205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6544,637,885,0.135197,149,0.006452,102,0.083813,0.115254,190,0.090261,...,376,0.136628,0.424859,2.479096,0.250847,11,11,225,2291,0.098210
6545,4760,398,0.060800,36,0.006452,234,0.192276,0.587940,330,0.156770,...,390,0.141715,0.979899,2.479899,0.590452,11,11,342,2291,0.149280
6546,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,...,13,0.004724,1.000000,2.153846,0.153846,11,10,13,2291,0.005674
6547,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,...,13,0.004724,1.000000,2.153846,0.153846,11,1,13,2291,0.005674


In [58]:
# Per Merchant Mean Action Type (Treats Action Type not really Discrete Label)
df_merchant['merchant_monthly_mean_action_type'] = df.groupby(['seller_id', 'month'])['action_type'].transform('mean')
df_merchant

,seller_id,merchant action count,merchant action count ratio,merchant day count,merchant product diversity,merchant 1111 action count,merchant 1111 action count ratio,merchant 1111 activity ratio,merchant last week action count,merchant last week action count ratio,...,last month action count ratio,last month activity ratio,average age,average gender,month,day,merchant_monthly_action_count,month_total_action_count,monthly_action_count_ratio,merchant_monthly_mean_action_type
0,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,...,0.001453,0.285714,4.500000,0.357143,11,11,3,2291,0.001309,0.000000
1,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,...,0.001453,0.285714,4.500000,0.357143,11,7,3,2291,0.001309,0.000000
2,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,0.000363,0.022222,3.400000,0.266667,8,20,10,704,0.014205,0.300000
3,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,0.000363,0.022222,3.400000,0.266667,8,20,10,704,0.014205,0.300000
4,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,0.000363,0.022222,3.400000,0.266667,8,20,10,704,0.014205,0.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6544,637,885,0.135197,149,0.006452,102,0.083813,0.115254,190,0.090261,...,0.136628,0.424859,2.479096,0.250847,11,11,225,2291,0.098210,0.528889
6545,4760,398,0.060800,36,0.006452,234,0.192276,0.587940,330,0.156770,...,0.141715,0.979899,2.479899,0.590452,11,11,342,2291,0.149280,0.324561
6546,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,...,0.004724,1.000000,2.153846,0.153846,11,10,13,2291,0.005674,0.307692
6547,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,...,0.004724,1.000000,2.153846,0.153846,11,1,13,2291,0.005674,0.307692


# Gender Interaction Count per Merchant

In [59]:
# For Other Columns, modify the groupby condition
male_counts = df.groupby(['seller_id'])['gender'].apply(lambda x: (x == 1).sum())
female_counts = df.groupby(['seller_id'])['gender'].apply(lambda x: (x == 0).sum())
unknown_gender_count = df.groupby(['seller_id'])['gender'].apply(lambda x: (x == 2).sum())
gender_total_merchant_counts = pd.DataFrame({'merchant_male_count': male_counts, 'merchant_female_count': female_counts, 'merchant_unknown_gender_count': unknown_gender_count})
gender_total_merchant_counts

,merchant_male_count,merchant_female_count,merchant_unknown_gender_count
seller_id,,,
1,6,36,3
5,1,1,0
25,2,2,0
107,0,1,0
139,0,30,0
...,...,...,...
4882,16,7,0
4955,8,46,4
4957,1,2,0


In [60]:
df_merchant = df_merchant.join(gender_total_merchant_counts, on='seller_id')
df_merchant

,seller_id,merchant action count,merchant action count ratio,merchant day count,merchant product diversity,merchant 1111 action count,merchant 1111 action count ratio,merchant 1111 activity ratio,merchant last week action count,merchant last week action count ratio,...,average gender,month,day,merchant_monthly_action_count,month_total_action_count,monthly_action_count_ratio,merchant_monthly_mean_action_type,merchant_male_count,merchant_female_count,merchant_unknown_gender_count
0,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,...,0.357143,11,11,3,2291,0.001309,0.000000,1,11,2
1,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,...,0.357143,11,7,3,2291,0.001309,0.000000,1,11,2
2,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,0.266667,8,20,10,704,0.014205,0.300000,6,36,3
3,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,0.266667,8,20,10,704,0.014205,0.300000,6,36,3
4,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,0.266667,8,20,10,704,0.014205,0.300000,6,36,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6544,637,885,0.135197,149,0.006452,102,0.083813,0.115254,190,0.090261,...,0.250847,11,11,225,2291,0.098210,0.528889,170,689,26
6545,4760,398,0.060800,36,0.006452,234,0.192276,0.587940,330,0.156770,...,0.590452,11,11,342,2291,0.149280,0.324561,197,182,19
6546,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,...,0.153846,11,10,13,2291,0.005674,0.307692,2,11,0
6547,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,...,0.153846,11,1,13,2291,0.005674,0.307692,2,11,0


# Monthly Gender Action Count per Merchant

In [61]:

monthly_male_counts = df.groupby(['seller_id', 'month'])['gender'].apply(lambda x: (x == 1).sum())
monthly_female_counts = df.groupby(['seller_id', 'month'])['gender'].apply(lambda x: (x == 0).sum())
monthly_unknown_gender_count = df.groupby(['seller_id', 'month'])['gender'].apply(lambda x: (x == 2).sum())
monthly_gender_total_merchant_counts = pd.DataFrame({'month_merchant_male_count': monthly_male_counts, 'month_merchant_female_count': monthly_female_counts, 'month_merchant_unknown_gender_count': monthly_unknown_gender_count})
monthly_gender_total_merchant_counts

month_merchant_male_count  month_merchant_female_count  \
seller_id month                                                           
1         5                              1                            3   
          6                              1                            1   
          7                              1                           10   
          8                              1                            9   
          9                              2                           12   
...                                    ...                          ...   
4955      11                             7                           40   
4957      6                              1                            2   
4964      10                             0                            1   
4988      10                             0                            2   
          11                             2                            0   

                 month_merchant_unknown_gender_count  
seller_id month                                       
1         5                                        0  
          6                                        0  
          7                                        3  
          8                                        0  
          9                                        0  
...                                              ...  
4955      11                                       4  
4957      6                                        0  
4964      10                                       0  
4988      10                                       0  
          11                                       0  

[395 rows x 3 columns]

In [62]:
df_merchant = df_merchant.join(monthly_gender_total_merchant_counts, on=['seller_id', 'month'])
df_merchant

,seller_id,merchant action count,merchant action count ratio,merchant day count,merchant product diversity,merchant 1111 action count,merchant 1111 action count ratio,merchant 1111 activity ratio,merchant last week action count,merchant last week action count ratio,...,merchant_monthly_action_count,month_total_action_count,monthly_action_count_ratio,merchant_monthly_mean_action_type,merchant_male_count,merchant_female_count,merchant_unknown_gender_count,month_merchant_male_count,month_merchant_female_count,month_merchant_unknown_gender_count
0,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,...,3,2291,0.001309,0.000000,1,11,2,0,1,2
1,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,...,3,2291,0.001309,0.000000,1,11,2,0,1,2
2,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,10,704,0.014205,0.300000,6,36,3,1,9,0
3,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,10,704,0.014205,0.300000,6,36,3,1,9,0
4,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,10,704,0.014205,0.300000,6,36,3,1,9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6544,637,885,0.135197,149,0.006452,102,0.083813,0.115254,190,0.090261,...,225,2291,0.098210,0.528889,170,689,26,46,170,9
6545,4760,398,0.060800,36,0.006452,234,0.192276,0.587940,330,0.156770,...,342,2291,0.149280,0.324561,197,182,19,173,154,15
6546,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,...,13,2291,0.005674,0.307692,2,11,0,2,11,0
6547,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,...,13,2291,0.005674,0.307692,2,11,0,2,11,0


# Merchant Gender Count Ratio

In [63]:
gender_counts = df_merchant[['merchant_male_count', 'merchant_female_count', 'merchant_unknown_gender_count']].sum(axis=1)
male_ratio = df_merchant.apply(lambda row: row['merchant_male_count'] / gender_counts[row.name], axis=1)
female_ratio = df_merchant.apply(lambda row: row['merchant_female_count'] / gender_counts[row.name], axis=1)
unknown_gender_ratio = df_merchant.apply(lambda row: row['merchant_unknown_gender_count'] / gender_counts[row.name], axis=1)
df_merchant['merchant_male_count_ratio'] = male_ratio
df_merchant['merchant_female_count_ratio'] = female_ratio
df_merchant['merchant_unknown_count_ratio'] = unknown_gender_ratio
df_merchant

,seller_id,merchant action count,merchant action count ratio,merchant day count,merchant product diversity,merchant 1111 action count,merchant 1111 action count ratio,merchant 1111 activity ratio,merchant last week action count,merchant last week action count ratio,...,merchant_monthly_mean_action_type,merchant_male_count,merchant_female_count,merchant_unknown_gender_count,month_merchant_male_count,month_merchant_female_count,month_merchant_unknown_gender_count,merchant_male_count_ratio,merchant_female_count_ratio,merchant_unknown_count_ratio
0,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,...,0.000000,1,11,2,0,1,2,0.071429,0.785714,0.142857
1,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,...,0.000000,1,11,2,0,1,2,0.071429,0.785714,0.142857
2,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,0.300000,6,36,3,1,9,0,0.133333,0.800000,0.066667
3,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,0.300000,6,36,3,1,9,0,0.133333,0.800000,0.066667
4,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,0.300000,6,36,3,1,9,0,0.133333,0.800000,0.066667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6544,637,885,0.135197,149,0.006452,102,0.083813,0.115254,190,0.090261,...,0.528889,170,689,26,46,170,9,0.192090,0.778531,0.029379
6545,4760,398,0.060800,36,0.006452,234,0.192276,0.587940,330,0.156770,...,0.324561,197,182,19,173,154,15,0.494975,0.457286,0.047739
6546,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,...,0.307692,2,11,0,2,11,0,0.153846,0.846154,0.000000
6547,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,...,0.307692,2,11,0,2,11,0,0.153846,0.846154,0.000000


# Merchant Buy Penetration

# Number of Buys per Merchant

In [64]:

merchant_buy = pd.DataFrame({'merchant_buys': df.groupby(['seller_id'])['action_type'].apply(lambda x: (x == 2).sum())})
df_merchant = df_merchant.join(merchant_buy, on=['seller_id'])
df_merchant

,seller_id,merchant action count,merchant action count ratio,merchant day count,merchant product diversity,merchant 1111 action count,merchant 1111 action count ratio,merchant 1111 activity ratio,merchant last week action count,merchant last week action count ratio,...,merchant_male_count,merchant_female_count,merchant_unknown_gender_count,month_merchant_male_count,month_merchant_female_count,month_merchant_unknown_gender_count,merchant_male_count_ratio,merchant_female_count_ratio,merchant_unknown_count_ratio,merchant_buys
0,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,...,1,11,2,0,1,2,0.071429,0.785714,0.142857,0
1,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,...,1,11,2,0,1,2,0.071429,0.785714,0.142857,0
2,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,6,36,3,1,9,0,0.133333,0.800000,0.066667,0
3,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,6,36,3,1,9,0,0.133333,0.800000,0.066667,0
4,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,6,36,3,1,9,0,0.133333,0.800000,0.066667,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6544,637,885,0.135197,149,0.006452,102,0.083813,0.115254,190,0.090261,...,170,689,26,46,170,9,0.192090,0.778531,0.029379,144
6545,4760,398,0.060800,36,0.006452,234,0.192276,0.587940,330,0.156770,...,197,182,19,173,154,15,0.494975,0.457286,0.047739,35
6546,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,...,2,11,0,2,11,0,0.153846,0.846154,0.000000,2
6547,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,...,2,11,0,2,11,0,0.153846,0.846154,0.000000,2


# Buy Ratio per Merchant

In [65]:
df_merchant['merchant_buy_ratio'] = df_merchant['merchant_buys'] / df_merchant.groupby(['seller_id']).transform('size')
df_merchant

,seller_id,merchant action count,merchant action count ratio,merchant day count,merchant product diversity,merchant 1111 action count,merchant 1111 action count ratio,merchant 1111 activity ratio,merchant last week action count,merchant last week action count ratio,...,merchant_female_count,merchant_unknown_gender_count,month_merchant_male_count,month_merchant_female_count,month_merchant_unknown_gender_count,merchant_male_count_ratio,merchant_female_count_ratio,merchant_unknown_count_ratio,merchant_buys,merchant_buy_ratio
0,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,...,11,2,0,1,2,0.071429,0.785714,0.142857,0,0.000000
1,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,...,11,2,0,1,2,0.071429,0.785714,0.142857,0,0.000000
2,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,36,3,1,9,0,0.133333,0.800000,0.066667,0,0.000000
3,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,36,3,1,9,0,0.133333,0.800000,0.066667,0,0.000000
4,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,36,3,1,9,0,0.133333,0.800000,0.066667,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6544,637,885,0.135197,149,0.006452,102,0.083813,0.115254,190,0.090261,...,689,26,46,170,9,0.192090,0.778531,0.029379,144,0.162712
6545,4760,398,0.060800,36,0.006452,234,0.192276,0.587940,330,0.156770,...,182,19,173,154,15,0.494975,0.457286,0.047739,35,0.087940
6546,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,...,11,0,2,11,0,0.153846,0.846154,0.000000,2,0.153846
6547,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,...,11,0,2,11,0,0.153846,0.846154,0.000000,2,0.153846


# Gender Buy Count per Merchant

In [66]:
merchant_male_buy = df[df.action_type == 2].groupby(['seller_id'])['gender'].apply(lambda x: (x == 1).sum()) 
merchant_female_buy = df[df.action_type == 2].groupby(['seller_id'])['gender'].apply(lambda x: (x == 0).sum())
merchant_unknown_buy = df[df.action_type == 2].groupby(['seller_id'])['gender'].apply(lambda x: (x == 2).sum())
gender_buys = pd.DataFrame({'merchant_male_buy_count': merchant_male_buy, 'merchant_female_buy_count': merchant_female_buy, 'merchant_unknown_buy_count': merchant_unknown_buy})
df_merchant = df_merchant.join(gender_buys, on=['seller_id'])
df_merchant

,seller_id,merchant action count,merchant action count ratio,merchant day count,merchant product diversity,merchant 1111 action count,merchant 1111 action count ratio,merchant 1111 activity ratio,merchant last week action count,merchant last week action count ratio,...,month_merchant_female_count,month_merchant_unknown_gender_count,merchant_male_count_ratio,merchant_female_count_ratio,merchant_unknown_count_ratio,merchant_buys,merchant_buy_ratio,merchant_male_buy_count,merchant_female_buy_count,merchant_unknown_buy_count
0,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,...,1,2,0.071429,0.785714,0.142857,0,0.000000,NaN,NaN,NaN
1,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,...,1,2,0.071429,0.785714,0.142857,0,0.000000,NaN,NaN,NaN
2,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,9,0,0.133333,0.800000,0.066667,0,0.000000,NaN,NaN,NaN
3,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,9,0,0.133333,0.800000,0.066667,0,0.000000,NaN,NaN,NaN
4,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,9,0,0.133333,0.800000,0.066667,0,0.000000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6544,637,885,0.135197,149,0.006452,102,0.083813,0.115254,190,0.090261,...,170,9,0.192090,0.778531,0.029379,144,0.162712,33.0,108.0,3.0
6545,4760,398,0.060800,36,0.006452,234,0.192276,0.587940,330,0.156770,...,154,15,0.494975,0.457286,0.047739,35,0.087940,17.0,16.0,2.0
6546,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,...,11,0,0.153846,0.846154,0.000000,2,0.153846,0.0,2.0,0.0
6547,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,...,11,0,0.153846,0.846154,0.000000,2,0.153846,0.0,2.0,0.0


# Gender buy Ratio per Merchant

In [67]:
df_merchant['merchant_male_buy_ratio'] = df_merchant['merchant_male_buy_count'] / df_merchant['merchant_buys']
df_merchant['merchant_female_buy_ratio'] = df_merchant['merchant_female_buy_count'] / df_merchant['merchant_buys']
df_merchant['merchant_unknown_buy_ratio'] = df_merchant['merchant_unknown_buy_count'] / df_merchant['merchant_buys']
df_merchant

,seller_id,merchant action count,merchant action count ratio,merchant day count,merchant product diversity,merchant 1111 action count,merchant 1111 action count ratio,merchant 1111 activity ratio,merchant last week action count,merchant last week action count ratio,...,merchant_female_count_ratio,merchant_unknown_count_ratio,merchant_buys,merchant_buy_ratio,merchant_male_buy_count,merchant_female_buy_count,merchant_unknown_buy_count,merchant_male_buy_ratio,merchant_female_buy_ratio,merchant_unknown_buy_ratio
0,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,...,0.785714,0.142857,0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
1,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,...,0.785714,0.142857,0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
2,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,0.800000,0.066667,0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
3,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,0.800000,0.066667,0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
4,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,0.800000,0.066667,0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6544,637,885,0.135197,149,0.006452,102,0.083813,0.115254,190,0.090261,...,0.778531,0.029379,144,0.162712,33.0,108.0,3.0,0.229167,0.750000,0.020833
6545,4760,398,0.060800,36,0.006452,234,0.192276,0.587940,330,0.156770,...,0.457286,0.047739,35,0.087940,17.0,16.0,2.0,0.485714,0.457143,0.057143
6546,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,...,0.846154,0.000000,2,0.153846,0.0,2.0,0.0,0.000000,1.000000,0.000000
6547,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,...,0.846154,0.000000,2,0.153846,0.0,2.0,0.0,0.000000,1.000000,0.000000


In [68]:

# Visualize rows with NaNs
nan_mask = df_merchant.isna().any(axis=1)
nan_rows = df_merchant[nan_mask]
nan_rows

,seller_id,merchant action count,merchant action count ratio,merchant day count,merchant product diversity,merchant 1111 action count,merchant 1111 action count ratio,merchant 1111 activity ratio,merchant last week action count,merchant last week action count ratio,...,merchant_female_count_ratio,merchant_unknown_count_ratio,merchant_buys,merchant_buy_ratio,merchant_male_buy_count,merchant_female_buy_count,merchant_unknown_buy_count,merchant_male_buy_ratio,merchant_female_buy_ratio,merchant_unknown_buy_ratio
0,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,...,0.785714,0.142857,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,...,0.785714,0.142857,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,0.800000,0.066667,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,0.800000,0.066667,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,0.800000,0.066667,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6504,4282,60,0.009166,31,0.006452,0,0.000000,0.000000,0,0.000000,...,0.283333,0.000000,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
6519,139,30,0.004583,19,0.006452,0,0.000000,0.000000,0,0.000000,...,1.000000,0.000000,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
6525,831,14,0.002139,8,0.006452,5,0.004108,0.357143,5,0.002375,...,0.928571,0.000000,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
6526,644,40,0.006111,28,0.006452,0,0.000000,0.000000,0,0.000000,...,0.925000,0.000000,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
condition = df_merchant['merchant_buys'] == 0
df_merchant.loc[condition, ['merchant_male_buy_count', 'merchant_female_buy_count', 'merchant_unknown_buy_count', 'merchant_male_buy_ratio', 'merchant_female_buy_ratio', 'merchant_unknown_buy_ratio']] = 0
df_merchant

,seller_id,merchant action count,merchant action count ratio,merchant day count,merchant product diversity,merchant 1111 action count,merchant 1111 action count ratio,merchant 1111 activity ratio,merchant last week action count,merchant last week action count ratio,...,merchant_female_count_ratio,merchant_unknown_count_ratio,merchant_buys,merchant_buy_ratio,merchant_male_buy_count,merchant_female_buy_count,merchant_unknown_buy_count,merchant_male_buy_ratio,merchant_female_buy_ratio,merchant_unknown_buy_ratio
0,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,...,0.785714,0.142857,0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
1,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,...,0.785714,0.142857,0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
2,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,0.800000,0.066667,0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
3,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,0.800000,0.066667,0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
4,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,0.800000,0.066667,0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6544,637,885,0.135197,149,0.006452,102,0.083813,0.115254,190,0.090261,...,0.778531,0.029379,144,0.162712,33.0,108.0,3.0,0.229167,0.750000,0.020833
6545,4760,398,0.060800,36,0.006452,234,0.192276,0.587940,330,0.156770,...,0.457286,0.047739,35,0.087940,17.0,16.0,2.0,0.485714,0.457143,0.057143
6546,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,...,0.846154,0.000000,2,0.153846,0.0,2.0,0.0,0.000000,1.000000,0.000000
6547,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,...,0.846154,0.000000,2,0.153846,0.0,2.0,0.0,0.000000,1.000000,0.000000


In [70]:

print(sum(df_merchant.isna().sum()))

0


In [71]:
# For Other Columns, modify the groupby condition
ageGroup_0_counts = df.groupby(['seller_id'])['age_range'].apply(lambda x: (x == 0).sum())
ageGroup_1_counts = df.groupby(['seller_id'])['age_range'].apply(lambda x: (x == 1).sum())
ageGroup_2_counts = df.groupby(['seller_id'])['age_range'].apply(lambda x: (x == 2).sum())
ageGroup_3_counts = df.groupby(['seller_id'])['age_range'].apply(lambda x: (x == 3).sum())
ageGroup_4_counts = df.groupby(['seller_id'])['age_range'].apply(lambda x: (x == 4).sum())
ageGroup_5_counts = df.groupby(['seller_id'])['age_range'].apply(lambda x: (x == 5).sum())
ageGroup_6_counts = df.groupby(['seller_id'])['age_range'].apply(lambda x: (x == 6).sum())
ageGroup_7_counts = df.groupby(['seller_id'])['age_range'].apply(lambda x: (x == 7).sum())
ageGroup_8_counts = df.groupby(['seller_id'])['age_range'].apply(lambda x: (x == 8).sum())
gender_total_item_counts = pd.DataFrame({'merchant_ageGroup_1_counts': ageGroup_1_counts,'merchant_ageGroup_2_counts': ageGroup_2_counts,'merchant_ageGroup_3_counts': ageGroup_3_counts,'merchant_ageGroup_4_counts': ageGroup_4_counts,'merchant_ageGroup_5_counts': ageGroup_5_counts,
                                          'merchant_ageGroup_6_counts': ageGroup_6_counts,'merchant_ageGroup_7_counts': ageGroup_7_counts,'merchant_ageGroup_8_counts': ageGroup_8_counts, 'merchant_ageGroup_0_counts': ageGroup_0_counts})
df_merchant = df_merchant.join(gender_total_item_counts, on='seller_id')
df_merchant

,seller_id,merchant action count,merchant action count ratio,merchant day count,merchant product diversity,merchant 1111 action count,merchant 1111 action count ratio,merchant 1111 activity ratio,merchant last week action count,merchant last week action count ratio,...,merchant_unknown_buy_ratio,merchant_ageGroup_1_counts,merchant_ageGroup_2_counts,merchant_ageGroup_3_counts,merchant_ageGroup_4_counts,merchant_ageGroup_5_counts,merchant_ageGroup_6_counts,merchant_ageGroup_7_counts,merchant_ageGroup_8_counts,merchant_ageGroup_0_counts
0,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,...,0.000000,0,2,4,1,3,0,4,0,0
1,3791,14,0.002139,11,0.006452,1,0.000822,0.071429,3,0.001425,...,0.000000,0,2,4,1,3,0,4,0,0
2,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,0.000000,0,0,8,26,5,0,0,0,6
3,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,0.000000,0,0,8,26,5,0,0,0,6
4,1,45,0.006874,36,0.006452,0,0.000000,0.000000,0,0.000000,...,0.000000,0,0,8,26,5,0,0,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6544,637,885,0.135197,149,0.006452,102,0.083813,0.115254,190,0.090261,...,0.020833,0,278,276,84,42,37,6,0,162
6545,4760,398,0.060800,36,0.006452,234,0.192276,0.587940,330,0.156770,...,0.057143,0,88,94,71,25,15,2,2,101
6546,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,...,0.000000,0,2,2,2,2,0,0,0,5
6547,1200,13,0.001986,4,0.006452,4,0.003287,0.307692,12,0.005701,...,0.000000,0,2,2,2,2,0,0,0,5


In [72]:
df_merchant.to_csv('test_merchant_profile.csv',index=False)

# End of Added Features (James)

In [ ]:
#FEATURE 16: LDA FEATURES

In [ ]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# define dataset
X = df_merchant
y = df['label']
print(X.shape, y.shape)

model = LinearDiscriminantAnalysis()

In [ ]:
# define model evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# summarize result
print('Mean Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

In [ ]:
model.fit(X, y)

In [ ]:
lda = []
for index,row in df_merchant.iterrows():
    yhat = model.predict([row])
    lda.append(yhat[0])
lda[:20]

In [ ]:
df_lda = pd.DataFrame({'label' : df['label'], 'lda' : lda})
df_lda

In [ ]:
#FEATURE 17: PCA FEATURES

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
x = df_merchant.copy()
y = df_lda['label']
x.drop('seller_id',axis=1,inplace=True)

In [ ]:
x

In [ ]:
from sklearn.preprocessing import StandardScaler
x_scaled = StandardScaler().fit_transform(x)
x_scaled

In [ ]:
y

In [ ]:
x_scaled = pd.DataFrame(x_scaled, index=x.index, columns=x.columns)
x_scaled

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x_scaled)

In [ ]:
print(pca.explained_variance_ratio_)

In [ ]:
print(pca.singular_values_)

In [ ]:
#Principal DF
principalDF = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])

In [ ]:
principalDF

In [ ]:
#Final Df
finalDF = pd.concat([principalDF, df[['label']]], axis = 1)

In [ ]:
finalDF

In [ ]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)

targets = [0, 1]
colors = ['r', 'b']
for label, color in zip(targets,colors):
    indicesToKeep = finalDF['label'] == label
    ax.scatter(finalDF.loc[indicesToKeep, 'principal component 1']
               , finalDF.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid()

In [ ]:
df_lda = pd.DataFrame({'pc1' : finalDF['principal component 1'], 'pc2' : finalDF['principal component 2']})
df_lda